<a href="https://colab.research.google.com/github/UmarKhattab09/ETL-Youtube/blob/main/ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importing Libraries and Downloading the Libraries

In [ ]:
!pip install youtube_transcript_api==0.6.2

import requests
import json
import polars as pl
from youtube_transcript_api import YouTubeTranscriptApi

### Pulling Video ID'S
1. You need Channel ID which can be Extracted from Youtube Channel under View Page Source
2. You need an API KEY which is listed as `mykey` in params


In [ ]:
channel_id = "UCNB_OaI4524fASt8h0IL8dw"
url = 'https://www.googleapis.com/youtube/v3/search'
page_token = None
video_record_list=[]


In [ ]:
def getVideoRecords(response: requests.models.Response) -> list:
    """
        Function to extract YouTube video data from GET request response
    """

    video_record_list = []

    for raw_item in json.loads(response.text)['items']:

        # only execute for youtube videos
        if raw_item['id']['kind'] != "youtube#video":
            continue

        video_record = {}
        video_record['video_id'] = raw_item['id']['videoId']
        video_record['datetime'] = raw_item['snippet']['publishedAt']
        video_record['title'] = raw_item['snippet']['title']

        video_record_list.append(video_record)

    return video_record_list

In [ ]:
%%time
# extract video data across multiple search result pages
while page_token != 0:
    # define parameters for API call
    params = {"key": mykey, 'channelId': channel_id, 'part': ["snippet","id"], 'order': "date", 'maxResults':50, 'pageToken': page_token}
    # make get request
    response = requests.get(url, params=params)

    # append video records to list
    video_record_list += getVideoRecords(response)

    try:
        # grab next page token
        page_token = json.loads(response.text)['nextPageToken']
    except:
        # if no next page token kill while loop
        page_token = 0



CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 13.4 µs


In [ ]:
df = pl.DataFrame(video_record_list)
df.head()

video_id,datetime,title
str,str,str
"""GdxpXf9LUQY""","""2025-03-19T18:00:06Z""","""The Power of the Last 10 Night…"
"""mQTQTOW78cc""","""2025-03-19T16:01:03Z""","""Your Duty is More Than Paying …"
"""0RaVy5k9DFk""","""2025-03-19T14:00:02Z""","""Revert Issues | Mufti Menk | R…"
"""Nku70MkGf_w""","""2025-03-19T12:00:28Z""","""Everyone&#39;s Blessing is Uni…"
"""vUGmz7DiLLQ""","""2025-03-19T10:40:04Z""","""Predestined Wealth VS Hard Wor…"


### Add transcripts to DataFrame


In [ ]:
def extract_text(transcript: list) -> str:
    """
        Function to extract text from transcript dictionary
    """

    text_list = [transcript[i]['text'] for i in range(len(transcript))]
    return ' '.join(text_list)

#### The Cell Below will take Time.

In [ ]:
%%time
transcript_text_list = []

for i in range(len(df)):

    # try to extract captions
    try:
        transcript = YouTubeTranscriptApi.get_transcript(df['video_id'][i])
        transcript_text = extract_text(transcript)
    # if not available set as n/a
    except:
        transcript_text = "n/a"

    transcript_text_list.append(transcript_text)

In [ ]:
df = df.with_columns(pl.Series(name="transcript", values=transcript_text_list))
print(df.head())

shape: (5, 4)
┌─────────────┬──────────────────────┬──────────────────────────────┬──────────────────────────────┐
│ video_id    ┆ datetime             ┆ title                        ┆ transcript                   │
│ ---         ┆ ---                  ┆ ---                          ┆ ---                          │
│ str         ┆ str                  ┆ str                          ┆ str                          │
╞═════════════╪══════════════════════╪══════════════════════════════╪══════════════════════════════╡
│ GdxpXf9LUQY ┆ 2025-03-19T18:00:06Z ┆ The Power of the Last 10     ┆ Welcome to the final stretch │
│             ┆                      ┆ Night…                       ┆ a…                           │
│ mQTQTOW78cc ┆ 2025-03-19T16:01:03Z ┆ Your Duty is More Than       ┆ n/a                          │
│             ┆                      ┆ Paying …                     ┆                              │
│ 0RaVy5k9DFk ┆ 2025-03-19T14:00:02Z ┆ Revert Issues | Mufti Menk | ┆ n/a    

In [ ]:
# shape + unique values
print("shape:", df.shape)
print("n unique rows:", df.n_unique())
for j in range(df.shape[1]):
    print("n unique elements (" + df.columns[j] + "):", df[:,j].n_unique())

shape: (523, 4)
n unique rows: 515
n unique elements (video_id): 515
n unique elements (datetime): 515
n unique elements (title): 515
n unique elements (transcript): 14


In [ ]:
print("Total number of title characters:", sum(len(df['title'][i]) for i in range(len(df))))
print("Total number of transcript characters:", sum(len(df['transcript'][i]) for i in range(len(df))))

Total number of title characters: 30541
Total number of transcript characters: 100249


In [19]:
# change datetime to Datetime dtype
df = df.with_columns(pl.col('datetime').cast(pl.Datetime))
print(df.head())
df.write_csv("Youtube.csv")

shape: (5, 3)
┌─────────────┬─────────────────────┬─────────────────────────────────┐
│ video_id    ┆ datetime            ┆ title                           │
│ ---         ┆ ---                 ┆ ---                             │
│ str         ┆ datetime[μs]        ┆ str                             │
╞═════════════╪═════════════════════╪═════════════════════════════════╡
│ GdxpXf9LUQY ┆ 2025-03-19 18:00:06 ┆ The Power of the Last 10 Night… │
│ mQTQTOW78cc ┆ 2025-03-19 16:01:03 ┆ Your Duty is More Than Paying … │
│ 0RaVy5k9DFk ┆ 2025-03-19 14:00:02 ┆ Revert Issues | Mufti Menk | R… │
│ Nku70MkGf_w ┆ 2025-03-19 12:00:28 ┆ Everyone&#39;s Blessing is Uni… │
│ vUGmz7DiLLQ ┆ 2025-03-19 10:40:04 ┆ Predestined Wealth VS Hard Wor… │
└─────────────┴─────────────────────┴─────────────────────────────────┘
